In [1]:
from mlflow.tracking import MlflowClient

MLFLOW_TRACKING_URI = "sqlite:///mlflow.db"

client = MlflowClient(MLFLOW_TRACKING_URI)

In [2]:
client.list_experiments()

[<Experiment: artifact_location='./mlruns/0', experiment_id='0', lifecycle_stage='active', name='Default', tags={}>,
 <Experiment: artifact_location='./mlruns/1', experiment_id='1', lifecycle_stage='active', name='nyc-taxi-experiment', tags={}>]

In [3]:
client.create_experiment("my-cool-experiment")

'2'

In [8]:
from mlflow.entities import ViewType

runs = client.search_runs(
    experiment_ids='1',
    filter_string="metrics.rmse < 6.8",
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"],
)

In [9]:
for run in runs:
    print(f"Run ID: {run.info.run_id}", f"RMSE: {run.data.metrics['rmse']}")

Run ID: 2bc681d8744e4639ad883c9e10cb544c RMSE: 6.396623670205836
Run ID: 7bd6482d897e40448bf187dc17e901f3 RMSE: 6.4167028006893165
Run ID: 1f6b61b2a90949bca23fdc5acbc0cbc3 RMSE: 6.422102873543081
Run ID: 06ddfb8faf54444cace30c5d84cb5294 RMSE: 6.523179033040928
Run ID: d5691308ae034db88f0eefab1de42bbf RMSE: 6.524520389842001
